In [4]:
!wget -N https://files.rcsb.org/download/1BNA.pdb

--2020-04-21 17:04:42--  https://files.rcsb.org/download/1BNA.pdb
Resolving files.rcsb.org (files.rcsb.org)... 132.249.213.91
Connecting to files.rcsb.org (files.rcsb.org)|132.249.213.91|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/octet-stream]
Saving to: ‘1BNA.pdb’

1BNA.pdb                [ <=>                ]  76.02K  --.-KB/s    in 0.04s   

Last-modified header missing -- time-stamps turned off.
2020-04-21 17:04:43 (1.88 MB/s) - ‘1BNA.pdb’ saved [77841]



In [5]:
import nglview as nv
import pytraj as pt
traj = pt.load('1BNA.pdb')
view = nv.show_pytraj(traj)
view

_ColormakerRegistry()

NGLWidget()

In [18]:
from simtk.openmm.app import *
from simtk.openmm import *
from simtk.unit import *
from sys import stdout

print('Loading...')
pdb = PDBFile('1BNA.pdb')
forcefield = ForceField('amber99sb.xml', 'tip3p.xml')
modeller = Modeller(pdb.topology, pdb.positions)
print('Adding hydrogens...')
modeller.addHydrogens(forcefield)
print('Adding solvent...')
modeller.addSolvent(forcefield, model='tip3p', padding=1*nanometer)
print('Minimizing...')
system = forcefield.createSystem(modeller.topology, nonbondedMethod=PME)

Loading...
Adding hydrogens...
Adding solvent...
Minimizing...


In [19]:
integrator = VerletIntegrator(0.002*picoseconds)
#platform = Platform.getPlatformByName('CUDA')
simulation = Simulation(modeller.topology, system, integrator)#,platform)
simulation.context.setPositions(modeller.positions)
simulation.minimizeEnergy(maxIterations=100)
print('Saving...')
positions = simulation.context.getState(getPositions=True).getPositions()
PDBFile.writeFile(simulation.topology, positions, open('1BNA_minimized.pdb', 'w'))
print('Done')

Saving...
Done


In [20]:
import nglview as nv
import pytraj as pt
traj = pt.load('1BNA_minimized.pdb')
view = nv.show_pytraj(traj)
view

NGLWidget()

In [21]:
simulation.reporters.append(PDBReporter('1BNA_produced.pdb', 10000))
simulation.reporters.append(StateDataReporter(stdout, 10000, step=True,
        potentialEnergy=True, temperature=True))
simulation.step(100000)

#"Step","Potential Energy (kJ/mole)","Temperature (K)"
10000,-419930.6840762526,87.94914320863407
20000,-420490.5590762526,90.6108690854363
30000,-421010.1840762526,93.12129273870163
40000,-421154.9340762526,93.76126449499108
50000,-421210.4340762526,94.02108210410611
60000,-421475.1840762526,95.31738919911794
70000,-421508.6840762526,95.45622205798077
80000,-421568.4340762526,95.73329734933492
90000,-421781.4340762526,96.73299890794469
100000,-421766.6840762526,96.65776084885474


In [22]:
import nglview as nv
import pytraj as pt
traj = pt.load('1BNA_produced.pdb')
view = nv.show_pytraj(traj)
view

NGLWidget(max_frame=9)